In [2]:
from cesnet_datazoo.datasets import CESNET_QUIC22
from cesnet_datazoo.config import DatasetConfig, AppSelection, ValidationApproach

dataset = CESNET_QUIC22("~/datasets/CESNET-QUIC22/", size="XS")

common_params = {
    "dataset": dataset,
    "apps_selection": AppSelection.ALL_KNOWN,
    "train_period_name": "W-2022-44",
    "val_approach": ValidationApproach.SPLIT_FROM_TRAIN,
    "train_val_split_fraction": 0.2,
    "use_packet_histograms": True,
}
dataset_config = DatasetConfig(**common_params)
dataset.set_dataset_config_and_initialize(dataset_config)
train_dataframe = dataset.get_train_df(flatten_ppi=True)
val_dataframe = dataset.get_val_df(flatten_ppi=True)
test_dataframe = dataset.get_test_df(flatten_ppi=True)

Loading data from dataloader


100%|██████████| 8162/8162 [00:08<00:00, 979.74it/s] 


Loading data from dataloader


100%|██████████| 192/192 [00:04<00:00, 47.79it/s]


Loading data from dataloader


100%|██████████| 1247/1247 [00:08<00:00, 151.52it/s]


In [3]:
for i in train_dataframe.loc[0].items():
    print(i)

('IPT_1', 0.0)
('IPT_2', 3.0)
('IPT_3', 0.0)
('IPT_4', 0.0)
('IPT_5', 111.0)
('IPT_6', 0.0)
('IPT_7', 0.0)
('IPT_8', 5.0)
('IPT_9', 1.0)
('IPT_10', 0.0)
('IPT_11', 0.0)
('IPT_12', 1.0)
('IPT_13', 0.0)
('IPT_14', 5.0)
('IPT_15', 1.0)
('IPT_16', 3.0)
('IPT_17', 1575.0)
('IPT_18', 1.0)
('IPT_19', 0.0)
('IPT_20', 26.0)
('IPT_21', 13.0)
('IPT_22', 29.0)
('IPT_23', 28.0)
('IPT_24', 1.0)
('IPT_25', 15.0)
('IPT_26', 4593.0)
('IPT_27', 1.0)
('IPT_28', 70.0)
('IPT_29', 0.0)
('IPT_30', 0.0)
('DIR_1', 1.0)
('DIR_2', -1.0)
('DIR_3', -1.0)
('DIR_4', -1.0)
('DIR_5', -1.0)
('DIR_6', -1.0)
('DIR_7', -1.0)
('DIR_8', 1.0)
('DIR_9', 1.0)
('DIR_10', -1.0)
('DIR_11', -1.0)
('DIR_12', 1.0)
('DIR_13', -1.0)
('DIR_14', 1.0)
('DIR_15', 1.0)
('DIR_16', 1.0)
('DIR_17', 1.0)
('DIR_18', 1.0)
('DIR_19', -1.0)
('DIR_20', 1.0)
('DIR_21', -1.0)
('DIR_22', -1.0)
('DIR_23', 1.0)
('DIR_24', 1.0)
('DIR_25', -1.0)
('DIR_26', 1.0)
('DIR_27', -1.0)
('DIR_28', -1.0)
('DIR_29', -1.0)
('DIR_30', -1.0)
('SIZE_1', 1232.0)
('SIZE_2

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score

from dataclasses import dataclass
import random

from pyqlearning.q_learning import QLearning

import numpy as np

@dataclass
class State_key:
    X_used : np.ndarray
    y_used : np.ndarray

    def __hash__(self):
        X_bytes = self.X_used.tobytes()
        y_bytes = self.y_used.tobytes()

        return hash((X_bytes, y_bytes))
    
    def __eq__(self, other):
        if not isinstance(other, State_key) or self.__hash__() != other.__hash__():
            return False
        return np.array_equal(self.X_used, other.X_used) and np.array_equal(self.y_used, other.y_used)

X = train_dataframe.drop(columns="APP").to_numpy()[:1000]
y = train_dataframe["APP"].to_numpy()[:1000]

X_test = test_dataframe.drop(columns="APP").to_numpy()[:100]
y_test = test_dataframe["APP"].to_numpy()[:100]

class Q(QLearning):
    # last_acc = 0
    # i_not_used = set()
    # X_used = np.ndarray()
    # y_used = np.ndarray()
    # clf = RandomForestClassifier()

    def update_state(self, state_key, action_key):
        return state_key

    def initialize(self, len, cols):
        self.X_used = np.ndarray(shape = (1, cols))
        self.y_used = np.ndarray(shape = (1, cols))
        self.last_acc = 0
        self.clf = RandomForestClassifier()
        self.i_not_used = set(range(len))

    def extract_possible_actions(self, state_key):
        return list(self.i_not_used)
    
    def select_action(self, state_key, next_action_list):
        epsilon_greedy_flag = bool(np.random.binomial(n=1, p=self.epsilon_greedy_rate))

        if epsilon_greedy_flag is False:
            action_key = random.choice(next_action_list)
        else:
            action_key = self.predict_next_action(state_key, next_action_list)

        self.i_not_used.remove(action_key)

        return action_key

    def train_clf(self, state_key):
        if self.t == 1:
            self.clf.fit(state_key.X_used[self.t - 1].reshape(1, -1), 
                        state_key.y_used[self.t - 1].reshape(1, -1))
        else:
            self.clf.fit(state_key.X_used[:(self.t - 1)], 
                        state_key.y_used[:(self.t - 1)])

    def test_acc(self, state_key):
        self.train_clf(state_key)

        predict_arr = self.clf.predict(X_test)

        return f1_score(y_test, predict_arr, average='weighted')

    def observe_reward_value(self, state_key, action_key):
        state_key.X_used[self.t - 1] = X[action_key]
        state_key.y_used[self.t - 1] = y[action_key]

        cur_acc = self.test_acc(state_key)
        reward = cur_acc - self.last_acc
        self.last_acc = cur_acc

        return reward

q = Q()
q.epsilon_greedy_rate = 0.1
q.alpha_value = 0.9
q.gamma_value = 0.9

print(y.shape)

iters = 100

for i in range(200):
    print(f"iter {i}\n")
    q.initialize(len(X), X.shape[1])
    state_key = State_key(np.ndarray(shape = (iters, X.shape[1])), 
                          np.ndarray(shape = (iters,)))
    
    q.learn(state_key, iters)
    predict_arr = q.clf.predict(X_test)
    print(f"q_learning_acc: {accuracy_score(y_test, predict_arr):.4f}")

clf = RandomForestClassifier()
clf.fit(X, y)
predict_arr = clf.predict(X_test)
print(f"base_model_acc: {accuracy_score(y_test, predict_arr):.4f}")


(1000,)
iter 0

q_learning_acc: 0.0600
iter 1

q_learning_acc: 0.0500
iter 2

q_learning_acc: 0.0000
iter 3

q_learning_acc: 0.0300
iter 4



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklea

q_learning_acc: 0.0300
iter 5

q_learning_acc: 0.0500
iter 6

q_learning_acc: 0.0000
iter 7

q_learning_acc: 0.0000
iter 8

q_learning_acc: 0.0200
iter 9



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklea

q_learning_acc: 0.0300
iter 10

q_learning_acc: 0.1000
iter 11

q_learning_acc: 0.0000
iter 12

q_learning_acc: 0.0100
iter 13



...

q_learning_acc: 0.0600
iter 198



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


q_learning_acc: 0.2900
iter 199



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


q_learning_acc: 0.2700
base_model_acc: 0.4900
